# Initial Imports


---



In [0]:
import pandas as pd
import numpy as np
import html5lib
pd.set_option( 'display.notebook_repr_html', False )

from IPython.display import HTML # useful for snippets
#  e.g. HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
from IPython.display import Image 
#  e.g. Image(filename='holt-winters-equations.png', embed=True) # url= also works
from IPython.display import YouTubeVideo
#  e.g. YouTubeVideo('1j_HxD4iLn8', start='43', width=600, height=400)
from IPython.core import page
get_ipython().set_hook('show_in_pager', page.as_hook(page.display_page), 0)

#  Generate PLOTS inside notebook, "inline" generates static png:
%matplotlib inline   
#          "notebook" argument allows interactive zoom and resize.


# note: https cannot be read by lxml

data_folder = 'data/'



#**Data Fetching**


---


## Scrapping SEC 13F Financial Report Filings

We want to know what asset managers own which stocks. The U.S. Securities and Exchange Commission (SEC) 13F Financial Report Filings provide the '13F-type' investment holdings (such as stocks) of US asset managers with at least $100M in assets. We can download an index of URLs to 13F Filings from the SEC's Electronic Data Gathering, Analysis, and Retrieval system (EDGAR). See the [SEC's website](https://www.sec.gov/edgar.shtml) for more information on EDGAR.

master.tsv is an index of Q3 13F financial report filings from 2013 to 2018 downloaded from the SEC's EDGAR system. We can use this index access and download each report via a url.

We reformat the master.tsv index attributes for ease of use, and filter for a specific type of 13F financial report called the 13F-HR which contains relevant information about institutional asset manager investment portfolios (how much of each investment they own).

In [0]:
master_index_df = pd.read_table(data_folder + 'master.tsv', sep='|', index_col=False, 
                 names=['CIK', 'Company Name', 'Filing Type', 'Filing Date', 'Filing URL .txt', 'Filing URL .html'], encoding='latin-1')
index_13f_df = master_index_df.loc[master_index_df['Filing Type'] == '13F-HR']


Because the raw original index is costly to load and filter from, we save 13f_index.tsv as a version of the index filtered for our needs for future computation. 

In [0]:
pd.index_13f_df.to_csv(data_folder + '13f_index.tsv')

Moving forward, we use this subset of the original index stored in 13f_index.tsv when running analysis for efficiency.

In [0]:
# load 2013 to 2018 Q3 report URLs
complete_13F_index_df = pd.read_table(data_folder + '13f_index.tsv', sep=',', index_col=False, encoding='latin-1')

We perform a sanity check to confirm that we are working with the types of URLs intended

In [0]:
# inspect if URL to be parsed is valid
complete_13F_index_df['Filing URL .html'].iloc[0]

'edgar/data/1000097/0000919574-13-005176-index.html'

We download each investor's Q3 13F financial report filing from 2013 to 2018 from the URL provided by the filtered 13f_index.tsv  index. 

In the process of scraping this data, we also reformat the data for readabilility and add a column denoting the percentage weight of each investment in each portfolio.



We also perform some **Data Cleaning and Selection**  by filtering the data from the reports downloaded for information relevant to our analysis, such as the investor name, the data the information was published,  and the name and value of each major investment. 

In [0]:
# initialize empty list to store dataframes from different investors (to be appended later)
appended_data = []

# loop through all reports, filter relevant data, create normalized dataframes per investor, add to list of dataframes to be appended
for index, row in complete_13F_index_df.iterrows():
     
    
    # need to parse initial html file for name of html file with investment data
    url = 'https://www.sec.gov/Archives/' + row['Filing URL .html'] #.iloc[index]
    page = pd.read_html( url )
    df = page[0]
    if len(df) < 5:
        print('skip: ' + url)
        continue
    table_url_suffix = df[2].iloc[4]

    report_suffix = row['Filing URL .html'] 
    investor = row['Company Name'] 
    date = row['Filing Date'] 
    
    ### SET TO RETURN TOP 20 STOCKS PER INVESTOR (BY SIZE OF INVESTMENT)
    num_stocks_returned = 20

    stem = 'http://www.sec.gov/Archives/'
    xml_suffix = '/xslForm13F_X01/'

    report_suffix = report_suffix.replace('-index.html', '')
    report_suffix = report_suffix.replace('-', '')

    #  build URL to html file with investment data
    url = stem + report_suffix + xml_suffix + table_url_suffix
    print(url)
    
    # turn HTML file into dataframe
    page = pd.read_html( url )
    #  the last element of page contains relevant investement data
    df = page[-1]

    #  rename columns:
    df.columns = [ 'stock', 'class', 'cusip', 'usd', 'size', 'sh_prin', 'putcall', 'discret', 'manager', 'vote1', 'vote2', 'vote3']

    #  But first three rows are SEC labels, not data, 
    #  so delete them:
    df = df[3:]

    #  Start a new index from 0 instead of 3:
    df.reset_index( drop=True )


    #  Delete irrevelant columns:
    dflite = df.drop( df.columns[[1, 4, 5, 7, 8, 9, 10, 11]], axis=1 )

    #  usd needs float type since usd was read as string:
    dflite[['usd']] = dflite[['usd']].astype( float )
    #                  NOTE: int as type will fail for NaN

    #  Type change allows proper sort:
    dfusd = dflite.sort_values( by=['usd'], ascending=[False] )


    usdsum = sum( dfusd.usd )
    #  Portfolio total in USD:
    #usdsum


    #  New column for percentage of total portfolio:
    dfusd['pcent'] = np.round(( dfusd.usd / usdsum ) * 100, 2)


    # New column for date of report filling
    dfusd.insert(0, 'date', date)

    # New column for investor
    dfusd.insert(0, 'investor', investor)

    #  Dataframe per investor with top num_stocks_returned 
    appended_data.append(dfusd.head( num_stocks_returned ))

# show list of dataframes    
#appended_data


We merge investment records from different investors together into a single dataframe, and will later save the final cleaned result to a CSV file locally because the data is time consuming to recompute.

In [0]:
# Concat investor dataframes together
appended_data = pd.concat(appended_data, axis=0)

## Loading CUSIP to Stock Symbol Mapping from cnfails Data
We want to annotate each investment stock with an industry standardized stock symbol (for example APPL for Apple Inc) to enable us to consistently clean and organize the financial records, as well as easily augment additional stock related data by the Stock Symbol attribute.

To do this, we load a mapping of the CUSIP attributes to Stock Symbols from the SEC's ( US Government Agency) cnfails dataset, dropping unrelevant attributes. 

In [0]:
# create mapping from cusip to symbol to allow Just Capital and 13F sample data to be joined
## load CUSIP to ticker data from "fail to deliver" data - we load several monthes of data to capture a greater number of unique investment managers
## note: "fail to deliver" data records the volume of transactions that a investment manager fails to deliver on
cusip_to_ticker_df_raw_data = pd.read_csv(data_folder + 'cnfails2018.txt', sep='|', index_col=False, encoding='latin-1' )
### drop duplicate entries for invesment managers
cusip_to_ticker_df = cusip_to_ticker_df_raw_data.drop_duplicates(subset=['CUSIP'], keep='first')
## drop unnessasary attributes from dataframe
cusip_to_ticker_df = cusip_to_ticker_df.drop(['SETTLEMENT DATE', 'DESCRIPTION'], axis=1)
## rename to make joining with 13F dataset easier
cusip_to_ticker_df = cusip_to_ticker_df.rename(index=str, columns={"CUSIP": "cusip", "SYMBOL": "symbol"})


## Loading Just Capital Compay Social Impact Scores

We want to be able to compare the social impact of different financial investements asset managers make. To do this, we load jc_companies.csv, an index of Just Capital Company social impact scores and rankings. This index gives a social impact score to the ~800 largest publicly traded companies, and is backed up by extensive research and a rigourously defended methodology. 

To learn more about the Just Capital Ranking Methodology, check out their [86 page report](https://com-justcapital-web-v2.s3.amazonaws.com/pdf/18May2018_JUST_Capital_2017_Ranking_Methodology.pdf) explaining their research methodology.

In [0]:
## load Just Capital company scoring and rankings of ~800 largest public (investable) companies 
jc_companies_df = pd.read_csv(data_folder + "jc_companies.csv")

## rename investment scoring dataframe ticker attribute to make joining data with 13F data easier
jc_companies_df = jc_companies_df.rename(index=str, columns={"ticker": "symbol"})

# Data Cleaning and Selection


---



We want to merge together the three datasets loaded above into a single dataframe for ease of analysis. 

We also want to know the portfolio-wide social impact of an investor, instead of just the impact of the investor's individual investements. To create this aggragate metric, we compute an additional attribute called 'weighted_score' that adjusts the social impact score of a particular investor's stock by the percentage allocation of that stock in the given investor's portfolio.

In [0]:
# for every investment manager asset holding entry in sec13f_sample_df, 
# if the "stock" matches a company scored by the Just Capital dataset, merge the datasets together
## do left outer join
merged_df = pd.merge(appended_data, cusip_to_ticker_df, how="inner", on="cusip")
merged_df = pd.merge(merged_df, jc_companies_df, how="inner", on = 'symbol')
## create weighted_score attribute
merged_df["weighted_score"] = merged_df['pcent']/100*merged_df['scores.overall']

# show merged dataset
merged_df.head()

We remove additional attributes that are not useful to our analysis, such as 'QUANTITY (FAILS)' from the cnfails dataset.

We convert the 'date' attribute into a datetime type and create a separate 'year' attribute based the date.

In [0]:
merged_df = merged_df.drop(['Unnamed: 0', 'QUANTITY (FAILS)', 'scores.worker.health'], axis=1)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year

We download this cleaned dataset result as 13f_results_13-18Q3_merged.csv to avoid having to rerun time consuming computations.

In [0]:
# Export as CSV file
merged_df.to_csv(data_folder + '13f_results_13-18Q3_merged.csv')

Moving forward, we run computations by directly loading the cleaned and saved 13f_results_13-18Q3_merged.csv result dataset. 

In [0]:
merged_df = pd.read_csv(data_folder + "13f_results_13-18Q3_merged.csv")


# Data Visualization

We will be using Plot.ly for data visualization. 

To prepare the data for visualization in Plotly, we select attributes and values of interest as shown below.

### We want to visualize the social impact of different asset manager investment portfolios

To do this, we create a dataframe for investor scores. 

This dataframe selects the following attributes: investor, year (when the investment was declared), and weighted_score (an aggragate score of the social impact of the stocks in an investors portfolio adjusted by portfolio composition).


In [0]:
# group merged dataset by investor, and sum the investment weighted_score entries by investor
investor_score_df = merged_df.groupby(['investor', 'year'])['weighted_score'].sum().reset_index()
investor_score_df = investor_score_df.sort_values('weighted_score', ascending=False)

Specifically, we are interested in showing a multi-year sample of data to show how social impact scores may have varied over time. 

We choose to visualize data for the years 2013, 2015, and 2017 since our dataset is from 2013 to 2018, we want to choose a sample set of evenly spaced out years, and because we are also performing Twitter data analysis for 2017. 

In [0]:
investor_score_13_15_17_df = investor_score_df.loc[(investor_score_df['year'] == 2013) | (investor_score_df['year'] == 2015) | (investor_score_df['year'] == 2017)]
#investor_score_13_15_17_df = investor_score_13_15_17_df.drop([16651])


We download the subset of data selected and returned in the investor_score_13_15_17_df dataframe, and generate a visualization using the Plot.ly Chart Studio tool.

Plot.ly Chart Studio allows us to create an interactive graph visualization of the [Portfolio Social Impact by Investor and Year](https://plot.ly/~mike.jiao/8/)

The 3-D Scatterplot shows the portfolio-wide social impact of institutional investors with over 100M in holders in a sample of three years between 2013 and 2018 (2013, 2015, and 2017). Portfolio-wide social impact is measured using a score computed as a weighted average of a portfolio’s individual stocks’ social impact. The score is computed using the portfolio weight percentage of each stock declared in SEC 13F filings, and individual stock social impact scores computed by Just Capital. 



In [0]:
# Export as CSV file
investor_score_13_15_17_df.to_csv(data_folder + 'investorsByYearScore_13_15_17.csv', index=False)

We choose to separately select and visualize Investor Portfolio Social Impact in 2017, since we are also analyzing Twitter data for 2017.

In [0]:
investor_score_17_df = investor_score_df.loc[(investor_score_df['year'] == 2017)]

We download the subset of data selected and returned in the investor_score_17_df dataframe, and generate a visualization using the Plot.ly Chart Studio tool.

Plot.ly Chart Studio allows us to create an interactive graph visualization of the [2017 Investor Portfolios by Social Impact](https://plot.ly/~mike.jiao/10/)

The 2-D Scatterplot focus in on year 2017 data from the previous 3-D Scatterplot.


In [0]:
# Export as CSV file
investor_score_17_df.to_csv(data_folder + 'investors2017Score.csv')

### We want to visualize the popularity of high social impact and low social impact stocks. 

Since we use Just Capital's Social Impact Company index as our metric for social impact, we first define high social impact stocks as the top 10% of companies in Just Capital's Social Impact Index, and low social impact stocks as the bottom 10% of companeis in Just Capital's Social Impact Index. 

The Just Capital Index includes 870 companies, so we select the top and bottom 87 companies. 

In [0]:
# top 10% social impact stocks
stock_score_top_df = jc_companies_df.nlargest(87, 'scores.overall')

# botton 10% social impact stocks
stock_score_bottom_df = jc_companies_df.nsmallest(87, 'scores.overall')

To find the popularity of the stocks, we count the number of times they appear in an investor's portfolio in 2017. 

We do this by extracting the set of high and low impact stocks from our augmented and cleaned 13F investor portfolio dataset (merged_df), and counting the occurance of unique stocks (via stock 'symbol').

In [36]:
popular_good_df = merged_df[merged_df["symbol"].isin(stock_score_top_df["symbol"])]
popular_good_df = popular_good_df.loc[(popular_good_df['year'] == 2017)]

popular_good_df = popular_good_df.groupby(['symbol', 'scores.overall'])['symbol'].agg({'Frequency':'count'})

popular_bad_df = merged_df[merged_df["symbol"].isin(stock_score_bottom_df["symbol"])]
popular_bad_df = popular_bad_df.loc[(popular_bad_df['year'] == 2017)]

popular_bad_df = popular_bad_df.groupby(['symbol', 'scores.overall'])['symbol'].agg({'Frequency':'count'})


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if __name__ == '__main__':


We download the subset of data selected and returned in the popular_good_df and popular_bad_df dataframes, and generate two visualization using the Plot.ly Chart Studio tool.

Plot.ly Chart Studio allows us to create an interactive graph visualization of the [Popularity of High Social Impact Stocks](https://plot.ly/~mike.jiao/12/) and [Popularity of Low Social Impact Stocks](https://plot.ly/~mike.jiao/14/) 

The heatmaps displays the social impact (scored by Just Capital) of each stock within the top or bottom 10% of Just Capital Social Impact Scores, as well as the number of large institutional investors (with at least 100M in US 13F type securities) that own that stock. 
More popular stocks are represented by lighter colours, whereas less popular stocks are represented with darker colours. 


In [0]:
# Export popularity of high social impact stocks as CSV file
popular_good_df.to_csv(data_folder + 'popular_good_df.csv')

# Export popularity of low social impact stocks as CSV file
popular_bad_df.to_csv(data_folder + 'popular_bad_df.csv')


### We want to visualize a comparision of investor portfolio social impact in 2013 compared to 2017.

Since we have data on the highest and lowest scoring social impact investor portfolios in 2013 and 2017, we aim to observe not only the state of social impact in investor portfolios in 2017, but where the top and bottom 10% of investor portfolios in 2013 ended up in 2017 based on portfolio social impact score. 

We select the top 10% of investors in 2013, and also select the social impact data of those same investors in 2017.

We do the same for the bottom 10% of investors in 2013.




In [0]:
# select 2013 investor portfolio social impact scores
investor_score_13_df = investor_score_df.loc[(investor_score_df['year'] == 2013)]

# top 10% 2013 (810 investors)
investor_score_top_2013_df = investor_score_13_df.nlargest(810, 'weighted_score')

# 2017 investor scores, for the top social investors from 2013
top_2013_investors_in_2017_df = investor_score_17_df[investor_score_17_df["investor"].isin(investor_score_top_2013_df["investor"])]

# bottom 10% 2013
investor_score_bottom_2013_df = investor_score_13_df.nsmallest(810, 'weighted_score')

# 2017 investor scores, for the bottom social investors from 2013
bottom_2013_investors_in_2017_df = investor_score_17_df[investor_score_17_df["investor"].isin(investor_score_bottom_2013_df["investor"])]


Likewise, we are interested in where the top and bottom 10% of investor portfolios in 2017 were in 2013 based on portfolio social impact score.

We find the top 10% of investors in 2017, and select the social impact data of those same investors in 2013.

We do the same for the bottom 10% of investors in 2017.

In [0]:
# top 10% 2017 (732 investors)
investor_score_top_2017_df = investor_score_17_df.nlargest(732, 'weighted_score')
# bottom 10% 2017 
investor_score_bottom_2017_df = investor_score_17_df.nsmallest(732, 'weighted_score')

#where were the 2017 bottom social investors in 2013
bottom_2017_investors_in_2013_df = investor_score_13_df[investor_score_13_df["investor"].isin(investor_score_bottom_2017_df["investor"])]

#where were the 2017 top social investors in 2013
top_2017_investors_in_2013_df = investor_score_13_df[investor_score_13_df["investor"].isin(investor_score_top_2017_df["investor"])]

We download the 6 subsets of data selected and returned in the following dataframes: 
* investor_score_17_df (already previously downloaded)
* top_2013_investors_in_2017_df
* bottom_2013_investors_in_2017_df
* bottom_2017_investors_in_2013_df
* top_2017_investors_in_2013_df





Using these datasets, we generate one visualization with 5 layers of data tracings using the Plot.ly Chart Studio tool.

Plot.ly Chart Studio allows us to create an interactive graph visualization of the [Investor Social Impact in 2013 vs 2017](https://plot.ly/~mike.jiao/25/)

The Scatterplot displays how the social impact of investor portfolios has changed from 2013 to 2017. The graph has 5 layers/traces that can be optionally displayed or removed from the plot by clicking the respective item in the Legend. These layers in order from top to bottom correspond with: 
1. All 2017 Investor Portfolio Social Impact Scores, 
2. 2017 Investor Portfolio Social Impact Scores for the top 10% of Investors by Portfolio Social Impact Score from 2013, 
3. 2017 Investor Portfolio Social Impact Scores for the bottom 10% of Investors by Portfolio Social Impact Score from 2013, 
4. 2013 Investor Portfolio Social Impact Scores for the top 10% of Investors by Portfolio Social Impact Score in 2017, and 
5. 2013 Investor Portfolio Social Impact Scores for the bottom 10% of Investors by Portfolio Social Impact Score in 2017.





Using these different layers, you can see how the Social Impact of the best and worst Social Impact Investors has changed, or not changed between 2013 and 2017. 


In [0]:
# Export top_2013_investors_in_2017_df as CSV file
top_2013_investors_in_2017_df.to_csv(data_folder + 'top_2013_investors_in_2017.csv')

# Export bottom_2013_investors_in_2017_df as CSV file
bottom_2013_investors_in_2017_df.to_csv(data_folder + 'bottom_2013_investors_in_2017.csv')

# Export bottom_2017_investors_in_2013_df as CSV file
bottom_2017_investors_in_2013_df.to_csv(data_folder + 'bottom_2017_investors_in_2013.csv')

# Export top_2017_investors_in_2013_df as CSV file
top_2017_investors_in_2013_df.to_csv(data_folder + 'top_2017_investors_in_2013.csv')